In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [44]:
#Canny Edge Detection
def getEdges(gray_img):
    # Define Canny Parameters
    kernel = 5
    lo = 50
    hi = 150
        
    gray_blur = cv2.GaussianBlur(gray_img,(kernel,kernel),0)
    edges = cv2.Canny(gray_blur,lo,hi)
    
    return edges

def getLines(mask_img,edge_img):
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = 1* np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10   # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 8 #minimum number of pixels making up a line
    max_line_gap = 6   # maximum gap in pixels between connectable line segments
    
    
    color_edges = np.dstack((edge_img, edge_img, edge_img)) 
    line_image = np.copy(mask_img)*0 # creating a blank to draw lines on
    line_image = np.dstack((line_image,line_image,line_image))
    
    lines = cv2.HoughLinesP(mask_img, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    
    return line_image

def overlayLines(img,line_image):
    return cv2.addWeighted(img,0.8,line_image,1,0)
    
def maskImg(img,vertices):
    
    ignore_mask_color = 255
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, np.int32(vertices), ignore_mask_color)
    masked_img = cv2.bitwise_and(img, mask)
    
    return masked_img

#Convert Grayscale Image To Be Compatable With RGB
def grayToThreeChannel(img):
    shape = img.shape
    ret = np.zeros([shape[0],shape[1],3])
    for i in range(0,3):
        ret[..., i] = img
        
    return ret

In [47]:
def rgb2Lanes(img):

    
    # Three Channel Color -> One Channel Gray
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    edges = getEdges(gray)
    
    imshape = img.shape[:2][::-1]
    # Mask Vertices Assuming A 1x1 Square
    generalVertices = np.array([[[.2,.92],[.46,.6],[.54,.6],[.8,.92]]])
    # Convert General Vertices In The Video's Resolution
    vertices = np.int32(np.multiply(generalVertices ,np.tile(imshape,(4,1))))

        
    masked_edges = maskImg(edges,vertices)
    
    lines_edges = getLines(masked_edges,edges)
    lines_overlayed = overlayLines(img,lines_edges)
    ret = lines_overlayed
    # One Channel Gray -> Three Channel Gray
    #ret = grayToThreeChannel(ret)
    
    return ret

In [48]:
new_clip_output = 'test_outputhough.mp4'
test_clip = VideoFileClip("test.mp4")
new_clip = test_clip.fl_image(rgb2Lanes) 
%time new_clip.write_videofile(new_clip_output, audio=False)

[MoviePy] >>>> Building video test_outputhough.mp4
[MoviePy] Writing video test_outputhough.mp4


100%|█████████▉| 681/682 [00:22<00:00, 29.78it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_outputhough.mp4 

CPU times: user 12 s, sys: 2.9 s, total: 14.9 s
Wall time: 23.8 s


In [49]:
HTML("""
<video width="640" height="300" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(new_clip_output))